In [1]:
import yaml
import wandb
import torchreid

config_path = "configs/im_osnet_ain_x1_0_softmax_256x128_amsgrad_cosine.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

# wandb.init(project=config['wandb']['project'],
#            entity=config['wandb']['entity'],
#            config=config)


/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
import torchreid
datamanager = torchreid.data.ImageDataManager(
    root            = config['data']['root'],
    sources         = config['data']['sources'],
    targets         = config['data']['targets'],
    height          = config['data']['height'],
    width           = config['data']['width'],
    batch_size_train= config['data']['batch_size'],
    batch_size_test = config['data']['batch_size'],
    transforms      = config['data']['transforms']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ color jitter
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6


In [3]:
model = torchreid.models.build_model(
    name        = config['model']['name'],
    num_classes = 2000 if 'tango2000' in config['data']['sources'] else datamanager.num_train_pids, #2000 for Tango
    loss        = config['loss']['name'],
    pretrained  = config['model']['pretrained'],
    # pooling     = config['model']['final_pooling'],
    **config['custom']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [4]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [4]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [5]:
engine.run(
    save_dir=f"log/{config['model']['name']}_{config['data']['sources'][0]}",
    max_epoch=60,
    eval_freq=5,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training
epoch: [1/60][50/101]	time 0.152 (0.168)	data 0.000 (0.006)	eta 0:16:52	loss 5.3795 (6.1840)	acc 12.5000 (5.6250)	lr 0.000300
epoch: [1/60][100/101]	time 0.150 (0.160)	data 0.000 (0.003)	eta 0:15:50	loss 3.7007 (5.3804)	acc 46.0938 (15.2422)	lr 0.000300
epoch: [2/60][50/101]	time 0.152 (0.154)	data 0.000 (0.004)	eta 0:15:09	loss 2.5597 (3.0453)	acc 66.4062 (54.6562)	lr 0.000300
epoch: [2/60][100/101]	time 0.149 (0.152)	data 0.000 (0.002)	eta 0:14:52	loss 2.1797 (2.7142)	acc 78.9062 (62.7500)	lr 0.000300
epoch: [3/60][50/101]	time 0.150 (0.154)	data 0.000 (0.004)	eta 0:14:56	loss 1.6747 (1.7839)	acc 86.7188 (86.0312)	lr 0.000300
epoch: [3/60][100/101]	time 0.151 (0.153)	data 0.000 (0.002)	eta 0:14:38	loss 1.6325 (1.7234)	acc 88.2812 (87.1016)	lr 0.000300
epoch: [4/60][50/101]	time 0.149 (0.154)	data 0.000 (0.004)	eta 0:14:38	loss 1.3953 (1.4312)	acc 95.3125 (93.9844)	lr 0.000300
epoch: [4/60][100/101]	time 0.149 (0.152)	data 0.000 (0.002)	eta 0:14:18	loss 1.3352 (1.415